## Generating all final ETM_files with mapping from external files

In [ ]:
#for all the dates from 01 to 31 - latest etm files 

import pandas as pd
import os

# Directories / folder
ETM_folder = "Latest_ETM_Data"
unmatched_stops = "Match"

os.makedirs(ETM_folder, exist_ok=True)
os.makedirs(unmatched_stops, exist_ok=True)

In [ ]:
#loop for all dates 

for date in range(1, 32):
    date_str = f"{date:02}"
    
    try:
        print(f"Processing for 2024-07-{date_str}...")
        
        ticket_df = pd.read_csv(f'ETM_Data/Ticket-Data-2024-07-{date_str}.csv')
        route_df = pd.read_csv('final_routes_directions.csv')
        
        ticket_df = ticket_df.merge(route_df[['Trip_Origin', 'Trip_Destination', 'Direction_Route']], 
                                    on=['Trip_Origin', 'Trip_Destination'], 
                                    how='left')
        ticket_df.rename(columns={'Direction_Route': 'Direction_route'}, inplace=True)

        # Process stop names and directions
        ticket_df['source'] = ticket_df['source'].str.upper().str.split().str.join(' ')
        ticket_df['destination'] = ticket_df['destination'].str.upper().str.split().str.join(' ')

        stops_df = pd.read_csv('latest_stops_centroid_numbering.csv')
        stops_df['DIRECTION'] = stops_df['DIRECTION'].str.upper().str.split().str.join(' ')
        stops_df['STOP NAME'] = stops_df['Stop Name'].str.upper().str.split().str.join(' ')

        # Map original and renamed stops
        final_stops_df = pd.read_csv('stops_mapping.csv')
        mapping_stops = final_stops_df.set_index('Stops')['Renamed Stops'].to_dict()

        ticket_df['source'] = ticket_df['source'].map(mapping_stops).fillna(ticket_df['source'])
        ticket_df['destination'] = ticket_df['destination'].map(mapping_stops).fillna(ticket_df['destination'])

        stops_df_unique = stops_df.drop_duplicates(subset=['STOP NAME', 'DIRECTION'])

        # Define merge_and_insert function
        def merge_and_insert(ticket_df, stops_df, stop_column, direction_column):
            ticket_df = ticket_df.merge(
                stops_df[['STOP.NO', 'STOP NAME', 'DIRECTION']],
                how='left',
                left_on=[stop_column, direction_column],
                right_on=['STOP NAME', 'DIRECTION']
            )
            stop_no_column = f"{stop_column}_stop_no"
            ticket_df = ticket_df.rename(columns={'STOP.NO': stop_no_column}).drop(columns=['STOP NAME', 'DIRECTION'])
            stop_no = ticket_df.pop(stop_no_column)
            stop_index = ticket_df.columns.get_loc(stop_column) + 1  
            ticket_df.insert(stop_index, stop_no_column, stop_no)
            return ticket_df

        # Insert stop numbers
        ticket_df = merge_and_insert(ticket_df, stops_df_unique, 'source', 'Direction_route')
        ticket_df = merge_and_insert(ticket_df, stops_df_unique, 'destination', 'Direction_route')

        # Process unmatched stops
        nodirection_stops_unique = stops_df[stops_df['DIRECTION'].isna()].drop_duplicates(subset=['STOP NAME'])

        def update_stop_no(ticket_df, stops_df, stop_column, stop_no_column):
            updated_df = ticket_df.merge(
                stops_df[['STOP.NO', 'STOP NAME']],
                how='left',
                left_on=stop_column,
                right_on='STOP NAME'
            )
            ticket_df.loc[ticket_df[stop_no_column].isna(), stop_no_column] = updated_df['STOP.NO']
            return ticket_df

        ticket_df = update_stop_no(ticket_df, nodirection_stops_unique, 'source', 'source_stop_no')
        ticket_df = update_stop_no(ticket_df, nodirection_stops_unique, 'destination', 'destination_stop_no')

        ticket_df = ticket_df.loc[:, ~ticket_df.columns.str.contains('^Unnamed')]
        ticket_df.to_csv(f'Temporary_File_{date_str}.csv', index=False)

        df = pd.read_csv(f'Temporary_File_{date_str}.csv', index_col=False)
        df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

        stops_df_unique = stops_df[['STOP NAME', 'CENTROID NUMBERING']].drop_duplicates(subset='STOP NAME')

        df = df.merge(stops_df_unique, left_on='source', right_on='STOP NAME', how='left')
        df = df.rename(columns={'CENTROID NUMBERING': 'source_zonal_centroid_number'})
        df = df.drop(columns=['STOP NAME'])
        cols = df.columns.tolist()
        source_idx = cols.index('source_stop_no')
        cols.insert(source_idx + 1, cols.pop(cols.index('source_zonal_centroid_number')))
        df = df[cols]

        df = df.merge(stops_df_unique, left_on='destination', right_on='STOP NAME', how='left')
        df = df.rename(columns={'CENTROID NUMBERING': 'destination_zonal_centroid_number'})
        df = df.drop(columns=['STOP NAME'])
        cols = df.columns.tolist()
        destination_idx = cols.index('destination_stop_no')
        cols.insert(destination_idx + 1, cols.pop(cols.index('destination_zonal_centroid_number')))
        df = df[cols]

        df.to_csv(f'{ETM_folder}/Final_Ticket_data_2024-07-{date_str}.csv', index=False)
        print(f"File saved to {ETM_folder}/Final_Ticket_data_2024-07-{date_str}.csv")

    except FileNotFoundError:
        print(f"File not found for 2024-07-{date_str}, skipping.")
    except Exception as e:
        print(f"An error occurred for 2024-07-{date_str}: {e}")
        